In [ ]:
# Скачаем и подключим PyGrib для считывания файлов ERA5

!pip install pygrib
import pygrib as grb

In [ ]:
# Подключим драйв, чтобы получить доступ к файлам, лежащим в нем

from google.colab import drive
drive.mount('/content/Data/') # выгружаем содержимое диска в папку /content/Data

Mounted at /content/Data/


In [ ]:
import pandas as pd
import numpy as np
import datetime
import math
from tqdm.notebook import tqdm

# Вытаскиваем таблицу пожаров из лежащего в драйве csv файла
path_to_fires = '/content/Data/MyDrive/Colab Notebooks/Baseline for MES/Dataset/fires_data1.csv'

fires_table = pd.read_csv(path_to_fires)
fires_table['dt'] = pd.to_datetime(fires_table.dt, format='%Y-%m-%d') # парсим даты

Создадим метод для вычисления ближайшей координаты в сетке к передаваемой широты и долготы. С помощью этого метода будем подтягивать к координате пожара данные по карте.

In [ ]:
def define_cell(lat,lon, lat_info = (41,81.5,0.1), lon_info = (19,169,0.1)):
  '''
  Метод, возвращающий ближайшую координату в сетке по передаваемой широте и долготе

  lat : передаваема широта
  lon : передаваемая долгота
  lat_info : информация о широте сетки (начало, конец, шаг)
  lon_info : инфомация о долготе сетки (начало, конец, шаг)
  find_index : флаг, благодаря которому можно будет получить индекс в решетке, вместо координат
  '''
  lat = min(lat, lat_info[1])
  lat = max(lat, lat_info[0])
  lon = min(lon, lon_info[1])
  lon = max(lon, lon_info[0])

 # таблица карты начинается с нижнего левого угла, а не с левого верхнего, как у массивов
  lat_idx = round((lat_info[1]-lat_info[0])/lat_info[2],5)
  lat_idx = round(lat_idx - round(round((lat-lat_info[0])/lat_info[2],5)))
  lon_idx = round(round((lon-lon_info[0])/lon_info[2],5))

  lat = round(round(lat/lat_info[2],5))*lat_info[2]
  lon = round(round(lon/lon_info[2],5))*lon_info[2]


  return ((round(lat,5),round(lon,5)),(lat_idx, lon_idx))      

In [ ]:
def extract_from_grib(grbindx, date):
  '''
  Метод, возвращающий список Параметров карты в конкретный день

  grbindx : grib.index объект, содержащий ключи дня месяца и года оригинального grib файла
  date : datetime объект искомого дня
  '''
  return grbindx.select(day = date.day, month = date.month, year = date.year)


def find_features(row, grbindx):
  '''
  Метод создает словарь с статистикой по карте в точке пожара

  row : DataRow object из DataFrame.iterrows()
  grbindx : grib.index объект, содержащий ключи дня месяца и года оригинального grib файла
  '''
  latlon, latlon_idx = define_cell(row['lat'], row['lon'])
  grib_messages = extract_from_grib(grbindx, row['dt'])
  params = {}
  params['lat'] = latlon[0]
  params['lon'] = latlon[1]
  params['lat_idx'] = latlon_idx[0]
  params['lon_idx'] = latlon_idx[1]
  for elem in grib_messages:
    params[elem.name] = elem.values[latlon_idx[0]][latlon_idx[1]]
  params['0d_fire'] = row['type_id']
  return params


def find_add_features(row, grbindx):
  '''
  Доп обработчик, принимающий не строку пандас таблицы, а кортеж с нужными данными (Для метода create_free_feat_dict)

  row : (lat, lon, date, type)
  grbindx : grib.index обект, содержащий часть ключей оригинального файла (day, month, year)
  '''
  latlon, latlon_idx = define_cell(row[0], row[1])
  grib_messages = extract_from_grib(grbindx, row[2])
  params = {}
  params['lat'] = latlon[0]
  params['lon'] = latlon[1]
  params['lat_idx'] = latlon_idx[0]
  params['lon_idx'] = latlon_idx[1]
  for elem in grib_messages:
    params[elem.name] = elem.values[latlon_idx[0]][latlon_idx[1]]
  params['0d_fire'] = row[3]
  return params

In [ ]:
def get_stats_over_period(table, since, to, fire_type=None, lat = (41,81.5), lon=(19,169)):
  '''
  Метод возвращает DataFrame, в котором содержатся отсеянные данные по определенным параметрам

  table -- пандас таблица, откуда брать данные
  since -- дата, с которой рассматривать пожары включительно
  to -- дата, до которой рассматривать пожары включительно
  fire_type -- список идентификаторов пожаров, которые нас интересуют. None если все пожары подходят
  lat -- кортеж из двух чисел как ограничение по широте
  lon -- кортеж из двух чисел как ограничение по долготе
  '''
  if fire_type is None:
    out = table.loc[(table['dt'] <= to) & 
                    (table['dt'] >= since) &
                    (table['lat'] >= lat[0]) &
                    (table['lat'] <= lat[1]) &
                    (table['lon'] >= lon[0]) &
                    (table['lon'] <= lon[1])]
  else:
    # на случай если нам нужны конкретные типы пожаров
    out = table.loc[(table['dt'] <= to) & 
                    (table['dt'] >= since) & 
                    table['type_id'].isin(fire_type) &
                    (table['lat'] >= lat[0]) &
                    (table['lat'] <= lat[1]) &
                    (table['lon'] >= lon[0]) &
                    (table['lon'] <= lon[1])]
  return out

In [ ]:
def create_feat_dict(path_togrib, pd_table, since_to, n_days = 3):
  '''
  Подготовка списка словарей с Фичами по пожарам, предоставленым в pd_table

  path_to_grib : Путь до grib файла
  pd_table : Пандас таблица с информацией о пожарах
  since_to : Кортеж из двух дататаймов (с какого по какой день смотреть информацию [)  )
  Инфа за весь сентябрь будет с 2020.09.1 - 2020.10.1
  n_days : Количество дней, для которых учитывается наличие пожара
  '''
  features = []
  # Вычреняем index объект, благодаря которому поиск будет происходить быстрее
  grbindx = grb.index(path_togrib, 'day', 'month','year')
  # Выделяем только нужные даты, чтобы ускорить поиск в ходе итераций
  searched_subsample = pd_table[(pd_table['dt'] >= since_to[0]) & (pd_table['dt'] < since_to[1] + datetime.timedelta(days=3))]
  # Выделяем использующиеся строки при обработке
  subsample = searched_subsample[searched_subsample['dt'] < since_to[1]]
  # Случайно выбираем не более 3000 строк (т.к. обработка оригинальных данных займет очень много времени)
  if subsample.shape[0] > 3000:
    subsample = subsample.sample(3000)
  for i, row in tqdm(subsample.iterrows(), total=subsample.shape[0]):
    # Создаем словарь с фичами к переданному пожару
    feat_dict = find_features(row,grbindx) 
    for j in range(1,n_days):  # получаю все пожары в этой точке на дни вперед
      fires = searched_subsample[(searched_subsample['dt'] == (row['dt'] + datetime.timedelta(days = j))) 
      & (searched_subsample['lon'] == feat_dict['lon'])
      & (searched_subsample['lat'] == feat_dict['lat'])]
      if fires.shape[0] == 0: 
        feat_dict['{}d_fire'.format(j)] = 11
      else:
        feat_dict['{}d_fire'.format(j)] = fires.iloc[0]['type_id']
    features.append(feat_dict)
  return features

In [ ]:
def create_3day_feat_dict(path_togrib, pd_table, since_to, n_days = 3):
  '''
  Подготовка списка словарей, ориентированных на пожар в третий день

  path_to_grib : Путь до grib файла
  pd_table : Пандас таблица с информацией о пожарах
  since_to : Кортеж из двух дататаймов (с какого по какой день смотреть информацию [)  )
  Инфа за весь сентябрь будет с [2020.09.1;2020.10.1)
  n_days : Количество дней, для которых учитывается наличие пожара
  '''
  features = []
  grbindx = grb.index(path_togrib, 'day', 'month','year')
  searched_subsample = pd_table[(pd_table['dt'] >= (since_to[0] - datetime.timedelta(days=3))) & (pd_table['dt'] < since_to[1] )]
  subsample = searched_subsample[searched_subsample['dt'] >= since_to[0]]
  if subsample.shape[0] > 1500:
    subsample = subsample.sample(1500)
  for i, row in tqdm(subsample.iterrows(), total=subsample.shape[0]):
    feat_dict = find_features(row,grbindx) # нахожу стату за текущий день = 0d_fire
    for j in range(1,n_days):  # получаю все пожары за предыдущие дни. -1d = 1d_fire;  -2d = 2d_fire...
      fires = searched_subsample[(searched_subsample['dt'] == (row['dt'] - datetime.timedelta(days = j))) 
      & (searched_subsample['lon'] == feat_dict['lon'])
      & (searched_subsample['lat'] == feat_dict['lat'])]
      if fires.shape[0] == 0:
        feat_dict['{}d_fire'.format(j)] = 11
      else:
        feat_dict['{}d_fire'.format(j)] = fires.iloc[0]['type_id']
    # сделаю перестановку значений по дням, чтобы было -2 -1 0й день, а не 0 -1 -2й
    values = [feat_dict['{}d_fire'.format(j)] for j in range(n_days-1,-1,-1)]
    for j in range(0,n_days):
      feat_dict['{}d_fire'.format(j)] = values[j]
    features.append(feat_dict)
  return features

In [ ]:
def create_free_feat_dict(path_togrib, pd_table, since_to, n_days = 3):
  '''
  Подготовка списка словарей, ориентированных на случайные точки по России (Для разбавления датасета)

  path_to_grib : Путь до grib файла
  pd_table : Пандас таблица с информацией о пожарах
  since_to : Кортеж из двух дататаймов (с какого по какой день смотреть информацию [)  )
  Инфа за весь сентябрь будет с 2020.09.1 - 2020.10.1
  n_days : Количество дней, для которых учитывается наличие пожара
  '''
  features = []
  grbindx = grb.index(path_togrib, 'day', 'month','year')
  searched_subsample = pd_table[(pd_table['dt'] >= since_to[0]) & (pd_table['dt'] < since_to[1] + datetime.timedelta(days=3))]
  subsample = searched_subsample[searched_subsample['dt'] < since_to[1]]
  for i in tqdm(range((since_to[1] - since_to[0]).days)):
    for k in range(40): # каждый день буду помечать 40 точек
      # генерирую рандомные координаты
      r_lat = round(np.random.uniform(46,71),1)
      r_lon = round(np.random.uniform(30,139),1)
      dt = since_to[0] + datetime.timedelta(days=i)
      f_type = 11
      # проверяю, а нет ли пожара на том месте
      row = searched_subsample[(searched_subsample['dt'] == dt) 
        & (searched_subsample['lon'] == r_lon)
        & (searched_subsample['lat'] == r_lat)]
      if(row.shape[0] != 0):
        f_type = row.iloc[0]['type_id']
      feat_dict = find_add_features((r_lat,r_lon,dt,f_type),grbindx)
      for j in range(1,n_days):  # получаю все пожары на дни вперед
        fires = searched_subsample[(searched_subsample['dt'] == (dt + datetime.timedelta(days = j))) 
        & (searched_subsample['lon'] == r_lon)
        & (searched_subsample['lat'] == r_lat)]
        if fires.shape[0] == 0:
          feat_dict['{}d_fire'.format(j)] = 11
        else:
          feat_dict['{}d_fire'.format(j)] = fires.iloc[0]['type_id']
      features.append(feat_dict)
  return features

In [ ]:
def fixate_all_data(path_to_grib, pd_table, since_to,path_to_write, i, feature_method):
  feats = feature_method(path_to_grib,pd_table,since_to,3)
  feats_pd = pd.DataFrame(feats)
  feats_pd.to_csv(path_to_write)
  print('{} file is written'.format(i))

In [ ]:
# Запускаем процесс создания csv файлов с фичами на каждый месяц 2020 года

for i in range(1,13):
  print('Month ', i)
  year = 2020
  time_from = datetime.datetime(year,i,1)                
  if i == 12:
    time_to = datetime.datetime(2021,1,1)
  else: 
    time_to = datetime.datetime(year,i+1,1) 
  
  fixate_all_data(f'/content/Data/MyDrive/Colab Notebooks/Baseline for MES/Dataset/{2020}-{i}-working.grib',
                  fires_table,
                  (time_from,time_to),
                  f'/content/Data/MyDrive/Colab Notebooks/Baseline for MES/Features/3dfeatures_{2020}_{i}.csv',
                  i,
                  create_3day_feat_dict)
  
  fixate_all_data(f'/content/Data/MyDrive/Colab Notebooks/Baseline for MES/Dataset/{2020}-{i}-working.grib',
                  fires_table,
                  (time_from,time_to),
                  f'/content/Data/MyDrive/Colab Notebooks/Baseline for MES/Features/freefeatures_{2020}_{i}.csv',
                  i,
                  create_free_feat_dict)
  
  fixate_all_data(f'/content/Data/MyDrive/Colab Notebooks/Baseline for MES/Dataset/{2020}-{i}-working.grib',
                  fires_table,
                  (time_from,time_to),
                  f'/content/Data/MyDrive/Colab Notebooks/Baseline for MES/Features/features_{2020}_{i}.csv',
                  i,
                  create_feat_dict)


